In [2]:
import torch
import numpy as np

In [23]:
size=tuple([2,3,2])
X=torch.randint(3,size=size)
A=torch.randint(3,size=size)
print("X mat")
print(X)
print("\n\nA mat")
print(A)

X mat
tensor([[[0, 1],
         [0, 1],
         [0, 2]],

        [[0, 0],
         [2, 0],
         [2, 2]]])


A mat
tensor([[[0, 2],
         [2, 2],
         [2, 0]],

        [[2, 1],
         [0, 1],
         [2, 0]]])


In [24]:
# out=torch.einsum('bnd,df->bnf',(X, A))
At=torch.transpose(A,1,2)
print(At)
out = torch.matmul(X,At)
print(out)

tensor([[[0, 2, 2],
         [2, 2, 0]],

        [[2, 0, 2],
         [1, 1, 0]]])
tensor([[[2, 2, 0],
         [2, 2, 0],
         [4, 4, 0]],

        [[0, 0, 0],
         [4, 0, 4],
         [6, 2, 4]]])


In [10]:
from sklearn.metrics import precision_score, f1_score, classification_report

y_true = [0, 1, 2, 0, 1, 2] # 3-class problem
y_pred = [0, 0, 1, 0, 0, 1] # we never predict '2'

mac=precision_score(y_true, y_pred, average='macro')
mic=precision_score(y_true, y_pred, average='micro') # no warning
non=precision_score(y_true, y_pred, average=None) 

print('mac={} mic={} non={}'.format(mac,mic,non))

mac=0.16666666666666666 mic=0.3333333333333333 non=[0.5 0.  0. ]
/dcs/pg20/u2085214/.conda/envs/disser/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
X=torch.tensor([0.22,-0.22],[0.33,-0.33],[0.8,-0,3])
A=torch.ones(size=(3,1))
print(X)
print(A)

TypeError: randint(): argument 'size' must be tuple of ints, but found element of type list at pos 2